In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

In [2]:
pip install --upgrade scipy networkx

  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: networkx
    Found existing installation: networkx 2.7.1
    Uninstalling networkx-2.7.1:
      Successfully uninstalled networkx-2.7.1
Note: you may need to restart the kernel to use updated packages.


# 1. Data

In [2]:
df_hero_net = pd.read_csv(r"hero-network.csv")
df_edges = pd.read_csv(r"edges.csv")
df_nodes = pd.read_csv(r"nodes.csv")

In [3]:
n_comics = df_nodes[df_nodes.type == 'comic'].count()
n_heros = df_nodes[df_nodes.type == 'hero'].count()
print(n_comics, n_heros)

node    12651
type    12651
dtype: int64 node    6439
type    6439
dtype: int64


## 1.1 Pre-processing 

In [4]:
df_hero_net["hero1"] = df_hero_net["hero1"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)
df_hero_net["hero2"] = df_hero_net["hero2"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)

In [98]:
df_nodes["node"] = df_nodes["node"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)
df_nodes["type"] = df_nodes["type"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)

In [119]:
df_nodes = df_nodes.replace('SPIDER-MAN/PETER PAR','SPIDER-MAN/PETER PARKER', regex=True)

In [5]:
df_edges["hero"] = df_edges["hero"].apply(lambda x: x[0:-1] if list(x)[-1] in [' ', '/'] else x)

In [6]:
hero = set(df_edges.hero)
hero_union = set(df_hero_net.hero1).union(set(df_hero_net.hero2))

diff = hero_union - hero.intersection(hero_union)
diff

{'SPIDER-MAN/PETER PAR'}

In [7]:
def jaccard_similarity(setA, setB):
    ''' 
    This function simply computes the Jaccard similarity from its definition.
    '''
    return len(set(setA).intersection(setB))/len(set(setA).union(setB))

In [8]:
for hero in set(df_edges.hero):
    for elem in diff:
        sim = jaccard_similarity(set(elem), set(hero))
        if sim >= 0.5:
            df_hero_net = df_hero_net.replace(elem, hero)

df_hero_net

,hero1,hero2
0,"LITTLE, ABNER",PRINCESS ZANDA
1,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
2,BLACK PANTHER/T'CHAL,PRINCESS ZANDA
3,"LITTLE, ABNER",PRINCESS ZANDA
4,"LITTLE, ABNER",BLACK PANTHER/T'CHAL
...,...,...
574462,COLOSSUS II/PETER RA,CALLISTO
574463,CALLISTO,ROGUE
574464,CALLISTO,CALIBAN
574465,CALIBAN,ROGUE


In [9]:
# test
hero = set(df_edges.hero)
hero_union = set(df_hero_net.hero1).union(set(df_hero_net.hero2))

diff = hero_union - hero.intersection(hero_union)
diff

set()

In [10]:
print(len(hero_union), len(hero.intersection(hero_union))) # all heroes' names in 'hero-network.csv' are now found in 'edges.csv'

6420 6420


## 1.2 First graph

In [11]:
G1 = nx.MultiGraph()

In [12]:
df_hero_net.apply(lambda row: G1.add_edge(row['hero1'], row['hero2']), axis=1)

0          0
1          0
2          0
3          1
4          1
          ..
574462    27
574463    22
574464     8
574465    11
574466     1
Length: 574467, dtype: int64

In [13]:
#drop sefl-loops
G1.remove_edges_from(nx.selfloop_edges(G1))

In [14]:
dicOfWeights = {}
for h1,h2 in df_hero_net.to_numpy():
    if h1 != h2:
        if (h1,h2) in dicOfWeights:
            dicOfWeights[(h1,h2)] += 1
        else:
            dicOfWeights[(h1,h2)] = 1


In [15]:
G1_weighted = nx.MultiGraph()

In [16]:
i = 0
for edge in list(G1.edges()):
    try:
        G1_weighted.add_edge(edge[0], edge[1], weight = 1/dicOfWeights[edge])
    except:
        G1_weighted.add_edge(edge[0], edge[1], weight = 1/dicOfWeights[(edge[1], edge[0])])


In [17]:
edges, weights = zip(*nx.get_edge_attributes(G1_weighted, 'weight').items())
for i in range(20):
    print((edges[i], weights[i]))

(('LITTLE, ABNER', 'PRINCESS ZANDA', 0), 0.2)
(('LITTLE, ABNER', 'PRINCESS ZANDA', 1), 0.2)
(('LITTLE, ABNER', 'PRINCESS ZANDA', 2), 0.2)
(('LITTLE, ABNER', 'PRINCESS ZANDA', 3), 0.2)
(('LITTLE, ABNER', 'PRINCESS ZANDA', 4), 0.2)
(('LITTLE, ABNER', 'PRINCESS ZANDA', 5), 0.2)
(('LITTLE, ABNER', 'PRINCESS ZANDA', 6), 0.2)
(('LITTLE, ABNER', 'PRINCESS ZANDA', 7), 0.2)
(('LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 0), 0.25)
(('LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 1), 0.25)
(('LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 2), 0.25)
(('LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 3), 0.25)
(('LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 4), 0.25)
(('LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 5), 0.25)
(('LITTLE, ABNER', "BLACK PANTHER/T'CHAL", 6), 0.25)
(('LITTLE, ABNER', 'CARNIVORE/COUNT ANDR', 0), 0.3333333333333333)
(('LITTLE, ABNER', 'CARNIVORE/COUNT ANDR', 1), 0.3333333333333333)
(('LITTLE, ABNER', 'CARNIVORE/COUNT ANDR', 2), 0.3333333333333333)
(('LITTLE, ABNER', 'CARNIVORE/COUNT ANDR', 3), 0.33333333333

## 1.3 Second graph

In [79]:
len(set(df_nodes['node']))


19090

In [120]:
attributes = {}
for elem,type in df_nodes.to_numpy():
    attributes[elem] = {'type' : type } 

In [121]:
len(attributes)

19087

In [122]:
len(G2.nodes)

19087

In [87]:
len(set(df_edges['hero']))+len(set(df_edges['comic']))

19090

In [123]:
G2 = nx.from_pandas_edgelist(df_edges, 'hero', 'comic')
nx.set_node_attributes(G2, attributes)

In [124]:
edges, types = zip(*nx.get_node_attributes(G2, 'type').items())

In [125]:
print(len(edges))
print(len(types))

19085
19085


In [20]:
edges, types = zip(*nx.get_node_attributes(G2, 'type').items())
for i in range(10):
    print(edges[i], types[i])

24-HOUR MAN/EMMANUEL hero
AA2 35 comic
3-D MAN/CHARLES CHAN hero
AVF 4 comic
AVF 5 comic
COC 1 comic
H2 251 comic
H2 252 comic
M/PRM 35 comic
M/PRM 36 comic


In [75]:
len(G2.nodes)

19087

In [21]:
df_edges

,hero,comic
0,24-HOUR MAN/EMMANUEL,AA2 35
1,3-D MAN/CHARLES CHAN,AVF 4
2,3-D MAN/CHARLES CHAN,AVF 5
3,3-D MAN/CHARLES CHAN,COC 1
4,3-D MAN/CHARLES CHAN,H2 251
...,...,...
96099,ZZZAX,H2 326
96100,ZZZAX,H2 327
96101,ZZZAX,M/CP 8/4
96102,ZZZAX,PM 47


In [22]:
edges = list(G2.edges)

In [23]:
edges[0:2]

[('24-HOUR MAN/EMMANUEL', 'AA2 35'), ('AA2 35', 'FROST, CARMILLA')]

# 2. Backend Implementation

# Functionality 3 - Shortest ordered Route

Input:

- The graph data
- A sequence of superheroes h = [h_2, ..., h_n-1]
- Initial node h_1 and an end node h_n
- N: denoting the top N heroes that their data should be considered

Output:

- The shortest walk of comics that you need to read to get from hero_1 to hero_n
Considerations: For this functionality, you need to implement an algorithm that returns the shortest walk that goes from node h_j to h_n, which visits in order the nodes in h. The choice of h_j and h_n can be made randomly (or if it improves the performance of the algorithm, you can also define it in any other way)

Important Notes:

- This algorithm should be run only on the second graph (G2).
- The algorithm needs to handle the case that the graph is not connected. Thus, only some of the nodes in h are reachable from h_1. In such a scenario, it is enough to let the program give in the output the string "There is no such path".
- Since we are dealing with walks, you can pass on the same node h_i more than once, but you have to preserve order. E.g., if you start from Spiderman to reach deadpool, and your path requires you to visit iron-man and colossus, you can go back to any comics any time you want, assuming that the order in which you visit the heroes is still the same.

In [126]:
att = nx.get_node_attributes(G2, "type")

In [117]:
len(att)

19085

In [118]:
len(G2.nodes)

19087

In [127]:
att['SPIDER-MAN/PETER PARKER']

KeyError: 'SPIDER-MAN/PETER PARKER'

In [29]:
nodes = set(nx.all_neighbors(G2,'3-D MAN/CHARLES CHAN' ))

In [30]:
nodes.add('3-D MAN/CHARLES CHAN')

In [37]:
l = list(nx.all_neighbors(G2,'ROSS, GEN. THADDEUS' ))
for i in l:
    nodes.add(i)


In [39]:
nodes.add('ROSS, GEN. THADDEUS')

In [62]:
len(nodes)

222

In [41]:
G2_new = G2.subgraph(nodes)

In [110]:
def top_N(df,N,G):
    df_new = df.groupby(['hero'])['hero'].count().reset_index(name="count")
    df_new= df_new.sort_values(by = 'count',ascending=False)
    df_new = df_new.head(N)
    nodes=[df_new['hero'][i] for i in df_new.index]
    print(nodes)
    
    new_nodes = set(nodes)
    for i in nodes:
        app = list(df_edges.loc[df_edges['hero']==i]['comic'])
        new_nodes.update(app)
        
    G_top_N = G.subgraph(new_nodes)
    return G_top_N

In [111]:
def shortest_path(G2,node,path,final_node,visited,shortest_one,att):
    if node == final_node and att[node]=='hero':
        path.append(node)
        if len(path) < len(shortest_one):
            shortest_one = path.copy()
        path.pop(len(path)-1)
        return shortest_one
    
    else:
        for new_node in list(nx.all_neighbors(G2, node)):
            if new_node not in visited:
                if att[new_node]=='comic':
                    path.append(new_node)
                visited.append(new_node)
                shortest_one = shortest_path(G2,new_node,path,final_node,visited,shortest_one,att)
                if att[new_node]=='comic':
                    path.pop(len(path)-1)
                visited.pop(len(visited)-1)   
        return shortest_one

In [112]:
def functionality_3(G2, N, h_1, h_2):
    if N:
        N = int(N)
        G2 = top_N(df_edges,N,G2)
    att = nx.get_node_attributes(G2, "type")
    if not nx.is_connected(G2):
        result = 'There is not such path because the graph is not connected'
        return result
    if h_1  not in list(G2.nodes) or h_2 not in list(G2.nodes):
        result = 'There is not such path'
        return result
    shortest_one = list(range(0,len(G2.nodes)))
    shortest_one = shortest_path(G2,h_1,[h_1],h_2,[h_1],shortest_one,att)
    return shortest_one

In [196]:
len(G2.nodes)

19087

In [45]:
N = input()

5


In [48]:
print(functionality_3(G2_new,N,'3-D MAN/CHARLES CHAN','ROSS, GEN. THADDEUS'))

['3-D MAN/CHARLES CHAN', 'H2 251', 'ROSS, GEN. THADDEUS']


In [115]:
print(functionality_3(G2,N,'THOR/DR. DONALD BLAK','CAPTAIN AMERICA'))

['SPIDER-MAN/PETER PARKER', 'CAPTAIN AMERICA', 'IRON MAN/TONY STARK', 'THING/BENJAMIN J. GR', 'THOR/DR. DONALD BLAK']


KeyError: 'SPIDER-MAN/PETER PARKER'

In [35]:
nx.shortest_path(G2, source='THOR/DR. DONALD BLAK', target='SPIDER-MAN/PETER PARKER', weight=None)

['THOR/DR. DONALD BLAK', 'A 11', 'SPIDER-MAN/PETER PARKER']

In [45]:
nx.shortest_path(G, source='3-D MAN/CHARLES CHAN', target='ROSS, GEN. THADDEUS', weight=None)

['3-D MAN/CHARLES CHAN', 'H2 251', 'ROSS, GEN. THADDEUS']

## Functionality 4 - Disconnecting Graphs (da rivedere/continuare)

Input:

- The graph data
- heroA: a superhero to which will relate sub-graph G_a
- heroB: a superhero to which will relate sub-graph G_b
- N: denoting the top N heroes that their data should be considered


Output:

- The minimum number of links (by considering their weights) required to disconnect the original graph in two disconnected subgraphs: G_a and G_b.

In [69]:
print(nx.is_connected(G1))
print(nx.is_connected(G2))

False
False


In [14]:
# nx.connected_components gets the list of components,
# max() command returns the largest one
components = nx.connected_components(G1)
largest_component = max(components, key=len)

In [16]:
# returns neighbors
sorted(list(G1.neighbors('JOCASTA')))[:10]

['3-D MAN/CHARLES CHAN',
 'AJAK/TECUMOTZIN [ETE',
 'ANGEL/WARREN KENNETH',
 'ANT-MAN II/SCOTT HAR',
 'ANT-MAN/DR. HENRY J.',
 'ANTOINETTE, MARIE',
 'ARABIAN KNIGHT/ABDUL',
 'ARBOGAST, BAMBI',
 'ASTROVIK, NORMA',
 'ATTUMA']

In [105]:
# returns all nodes reachable from source in G
sorted(list(nx.descendants(G1, 'JOCASTA')))[:10]

['24-HOUR MAN/EMMANUEL',
 '3-D MAN/CHARLES CHAN',
 '4-D MAN/MERCURIO',
 '8-BALL',
 'A',
 "A'YIN",
 'ABBOTT, JACK',
 'ABCISSA',
 'ABEL',
 'ABOMINATION | MUTANT']

In [15]:
# add N
def disconneting_graphs(G, heroA, heroB):
    G_a = []
    G_b = []

    for edge in list(G.edges()):
        if heroA in edge:
            G_a.append(edge)

    for edge in list(G.edges()):
        if heroB in edge:
            G_b.append(edge)

    return G_a, G_b, len(G_a) + len(G_b)

In [17]:
G_a, G_b, result = disconneting_graphs(G1, 'JOCASTA', '8-BALL')
print(result)

760


In [102]:
newG1 = G1.copy()
newG1.remove_edges_from(G_a)
newG1.remove_edges_from(G_b)

In [108]:
G1.number_of_edges() - newG1.number_of_edges()

760

In [106]:
print(len(list(newG1.neighbors('JOCASTA'))))
print(len(list(newG1.neighbors('8-BALL'))))


0
0
